In [1]:
import numpy as np
import sympy as sp
import galois

Consider a quantum system with Hilbert space dimension odd $d$, and the finite ring $\mathbb Z_d$. Vourdas' formalism does not work for even characteristic.

In [2]:
d = 3 # odd

F = galois.GF(d)
F.elements

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


GF([0, 1, 2], order=3)

A few utilities and the definition of the $\omega(\alpha)$.

In [3]:
II = sp.eye(d)

def tensorproduct(a, b):
    return sp.ImmutableMatrix((sp.tensorproduct(a, b)).reshape(d, d))

def lift(a):
    return np.int64(np.array(a))

def omega(a):
    return sp.exp(sp.I * 2 * sp.pi * lift(a) / d)

Define the finite Fourier transform.

In [4]:
FF = sp.zeros(d, d)
for m in F.elements:
    for n in F.elements:
        FF += omega(m * n) * tensorproduct(II[:, m], sp.adjoint(II[:, n]))
FF = FF / sp.sqrt(d)

In [5]:
FF

Matrix([
[sqrt(3)/3,                sqrt(3)/3,                sqrt(3)/3],
[sqrt(3)/3,  sqrt(3)*exp(2*I*pi/3)/3, sqrt(3)*exp(-2*I*pi/3)/3],
[sqrt(3)/3, sqrt(3)*exp(-2*I*pi/3)/3,  sqrt(3)*exp(2*I*pi/3)/3]])

In [6]:
(FF * sp.adjoint(FF)).applyfunc(sp.nsimplify) # sanity check!

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [7]:
(FF**4).applyfunc(sp.nsimplify)

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

Define the basic translation operators $Z$ and $X$.

In [8]:
Z = sp.zeros(d, d)
X = sp.zeros(d, d)
for n in F.elements:
    v = II[:, n]
    fv = FF * v
    Z += omega(n) * tensorproduct(v, sp.adjoint(v))
    X += omega(-n) * tensorproduct(fv, sp.adjoint(fv))
X = X.applyfunc(sp.nsimplify)

In [9]:
Z

Matrix([
[1,             0,              0],
[0, exp(2*I*pi/3),              0],
[0,             0, exp(-2*I*pi/3)]])

In [10]:
X

Matrix([
[0, 0, 1],
[1, 0, 0],
[0, 1, 0]])

In [11]:
(X**lift(F.elements[2])) * II[:,0] # sanity check

Matrix([
[0],
[0],
[1]])

Now define the general displacement operators $D(\alpha,\beta)$. Notice, that for odd order, the inverse of the element $2$ exists.

In [13]:
def D(a, b):
    return omega((-F.elements[2]**(-1)) * a * b) * (Z**lift(a)) * (X**lift(b))

In [14]:
D(F.elements[0], F.elements[1]) # sanity check

Matrix([
[0, 0, 1],
[1, 0, 0],
[0, 1, 0]])

In [15]:
D(F.elements[1], F.elements[2])

Matrix([
[            0, exp(-2*I*pi/3), 0],
[            0,              0, 1],
[exp(2*I*pi/3),              0, 0]])

From here, we can define the displaced parity operators and then the Wigner functions. No problema at all for odd dimensions.

In [16]:
P0 = (FF**2).applyfunc(sp.nsimplify)
def P(a,b):
    return D(F.elements[2]*a, F.elements[2]*b) * P0

In [17]:
def Wigner(theta, a, b):
    return sp.trace(theta * P(a,b))

In [18]:
def W(theta):
    w = sp.zeros(d, d)
    for i, a in enumerate(F.elements):
        for j, b in enumerate(F.elements):
            w[i,j] = Wigner(theta, a, b)
    return w

In [19]:
state = (II[:,0] + sp.I*II[:,1] + 2*II[:,2])/sp.sqrt(6)
S = tensorproduct(state, sp.adjoint(state))

In [20]:
W(S).applyfunc(sp.nsimplify).applyfunc(sp.N)

Matrix([
[ 0.166666666666667,  0.833333333333333, 0.666666666666667],
[-0.410683602522959, -0.166666666666667,  0.95534180126148],
[ 0.744016935856292, -0.166666666666667, 0.377991532071854]])

How about we try to construct those mutually unbiased bases? Vourdas uses the symplectic transformations he defines in chapter 4. Precisely, he defines $p$ mutually unbiased bases given by $$|\mathbb X(\nu); m\rangle = S(0,-1|1,\nu) |X; m\rangle; \quad v,m \in \mathbb Z_p.$$ Vourdas defines such transformations by their action on the operatros $X$ and $Z$. But we can use the position states to obtain the matrix representation of such unitary operators. In general we have for $\mu,\nu \in [\mathbb Z_d]^*$:
$$
\langle X; r| S(\kappa, \lambda | \mu, \nu) |X; n\rangle
= \frac{1}{d} G[-2^{-1} \mu \nu^{-1}; \mathbb Z_d] \times \omega[2^{-1} \lambda \nu^{-1} n^2 + 2^{-1} \mu^{-1} \nu^{-1} (r\nu -n)^2].
$$
Where $G[s, \mathbb Z_d]$ is the Gauss sum:
$$
G[s; \mathbb Z_d] = \sum_{\beta = 0}^{d-1} \omega(s \beta^2).
$$
So in particular
\begin{align*}
\langle X; r| S(0,-1|1,\nu) |X; n\rangle
&= \frac{1}{d} G[-2^{-1} \nu^{-1}; \mathbb Z_d] \times \omega[-2^{-1} \nu^{-1} n^2 + 2^{-1} \nu^{-1} (r\nu - n)^2] \\
&= \frac{1}{d} G[-2^{-1} \nu^{-1}; \mathbb Z_d] \times \omega[-2^{-1} \nu^{-1} \left(n^2 - (r\nu - n)^2\right)] \\
&= \frac{1}{d} G[-2^{-1} \nu^{-1}; \mathbb Z_d] \times \omega[2^{-1} \left(r^2\nu - 2rn\right)]
\end{align*}

In [21]:
def GaussSum(a):
    return sum([omega(a * beta**2) for beta in F.elements])

In [22]:
def Symp(nu):
    s = sp.zeros(d, d)
    two = F.elements[2]
    for i, r in enumerate(F.elements):
        for j, n in enumerate(F.elements):
            s[i,j] = GaussSum(-two**(-1) * nu**(-1)) * omega(two**(-1) * (r**2 * nu - two*r*n))
    return s / d

The MUBs are then given by the identity matrix $I$, the Fourier transform $F$ and the two symplectic transformations $S(0,-1|1,1)$ and $S(0,-1|1,2)$.

In [23]:
# II
# FF
S1 = Symp(F.elements[1]).applyfunc(sp.nsimplify)
S2 = Symp(F.elements[2]).applyfunc(sp.nsimplify)

In [28]:
II

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [29]:
FF

Matrix([
[sqrt(3)/3,                sqrt(3)/3,                sqrt(3)/3],
[sqrt(3)/3,  sqrt(3)*exp(2*I*pi/3)/3, sqrt(3)*exp(-2*I*pi/3)/3],
[sqrt(3)/3, sqrt(3)*exp(-2*I*pi/3)/3,  sqrt(3)*exp(2*I*pi/3)/3]])

In [27]:
S1

Matrix([
[      sqrt(3)*I/3,        sqrt(3)*I/3,        sqrt(3)*I/3],
[1/2 - sqrt(3)*I/6, -1/2 - sqrt(3)*I/6,        sqrt(3)*I/3],
[1/2 - sqrt(3)*I/6,        sqrt(3)*I/3, -1/2 - sqrt(3)*I/6]])

In [30]:
S2

Matrix([
[     -sqrt(3)*I/3,       -sqrt(3)*I/3,       -sqrt(3)*I/3],
[1/2 + sqrt(3)*I/6,       -sqrt(3)*I/3, -1/2 + sqrt(3)*I/6],
[1/2 + sqrt(3)*I/6, -1/2 + sqrt(3)*I/6,       -sqrt(3)*I/3]])

In [24]:
def verifyMUB(a,b):
    return sp.Abs((sp.adjoint(a) * b)[0])

In [25]:
verifyMUB(FF[:,2], S1[:,0])

sqrt(3)/3

The computed Wigner function coincides with Vourdas' calculation and the symplectic transformations do indeed yield MUBs. All good for odd prime dimensions.